In [1]:
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

In [3]:
from utils_training import Dataset_FV
from utils_training import FVTrainingData
from utils_training import evaluate_FVmodel

In [4]:
from behavior_model import FVBehaviorModel

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
# Replace 'your_file.csv' with the actual path to your CSV file
file_path = 'Synthetic_crash_scenarios.csv'
# file_path = '/content/drive/MyDrive/Colab Notebooks/Thesis Crash Generation/Synthetic_crash_scenarios.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

## Build the dataset

In [7]:
acc_min, acc_max, acc_interval =  -9.5, 4, 0.25
acc_std_dev = 0.07
Inputs, Inputs_Vl, Outputs, Weights = FVTrainingData(df)

In [8]:
## Load the train, val, test indexes

import pickle
with open('.\\utils_data\\Idx_dtw_02_split.pkl', 'rb') as f:
    list_train_val_test = pickle.load(f)
list_train_val_test = list_train_val_test[0]

Idx_train, Idx_val, Idx_test = list_train_val_test[0], list_train_val_test[1], list_train_val_test[2]

In [9]:
## Split the Inputs, Outputs, Weights based on train, val, test indexes

Inputs_train = [Inputs[i] for i in Idx_train]
Inputs_Vl_train = [Inputs_Vl[i] for i in Idx_train]
Outputs_train = [Outputs[i] for i in Idx_train]
Weights_train = [Weights[i] for i in Idx_train]

Inputs_val = [Inputs[i] for i in Idx_val]
Inputs_Vl_val = [Inputs_Vl[i] for i in Idx_val]
Outputs_val = [Outputs[i] for i in Idx_val]
Weights_val = [Weights[i] for i in Idx_val]

In [10]:
## Create dataset and dataloader

input_size = Inputs[0].shape[1] # Dimension of the input
batch_size = 64

# dataset = BCDataset(Inputs, Outputs, Weights)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_train = Dataset_FV(Inputs_train, Inputs_Vl_train, Outputs_train, Weights_train)
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = Dataset_FV(Inputs_val, Inputs_Vl_val, Outputs_val, Weights_val)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

## Model Training

In [11]:
## Network Parameters 
gru_hidden_size = 256 # Size of GRU's hidden layer
mlp_hidden_size = 128 # Size of MLP's hidden layer
output_size = int((acc_max - acc_min)/acc_interval) + 1

## Training Parameters
num_epochs = 500
learning_rate = 0.0001

## Instantiate model, loss function, and optimizer
model = FVBehaviorModel(input_size, gru_hidden_size, mlp_hidden_size, output_size).to(device)
# criterion = nn.KLDivLoss(reduction='batchmean').to(device)
criterion = nn.KLDivLoss(reduction='none').to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [12]:
best_val_loss = float('inf')
best_train_loss = float('inf')
best_trainval_loss = float('inf')
best_val_epoch = 0
best_train_epoch = 0
best_trainval_epoch = 0
best_val_model_path = f'.\\weights\\bc_fv_val_dtw_02.pth'
best_train_model_path = f'.\\weights\\bc_fv_train_dtw_02.pth'
best_trainval_model_path = f'.\\weights\\bc_fv_trainval_dtw_02.pth'

In [13]:
num_epochs = 4000

In [14]:
for epoch in range(num_epochs):
    model.train() # set model as training mode
    for inputs, inputs_vl, targets, sample_weights in dataloader_train:
        inputs = inputs.float().to(device)
        inputs_vl = inputs_vl.reshape(inputs_vl.shape[0],1,inputs_vl.shape[1])
        inputs_vl = inputs_vl.float().to(device)
        targets = targets.float().to(device)
        sample_weights = sample_weights.to(device)
        
        optimizer.zero_grad()
        log_probs = model(inputs,inputs_vl)
        ## calculate loss
        loss = criterion(log_probs, targets)
        weighted_loss = torch.sum(torch.sum(loss,axis=2) * sample_weights.reshape(-1,1))/torch.sum(sample_weights) ## loss per weight
        weighted_loss.backward()
        optimizer.step()
    
    ## Calculate the mean, min, max loss of this epoch
    model.eval()  # set model as evaluation mode
    ## loss of training dataset
    train_loss_mean, train_loss_min, train_loss_max = evaluate_FVmodel(model, criterion, dataloader_train, device)
    ## loss of validation dataset
    test_loss_mean, test_loss_min, test_loss_max = evaluate_FVmodel(model, criterion, dataloader_val, device)
    ## loss of combination
    trainval_loss_mean = (test_loss_mean*len(dataset_val) + train_loss_mean*len(dataset_train)) / (len(dataset_val)+len(dataset_train))

    print(f'Epoch {epoch+1}/{num_epochs}, Loss_mean: {round(train_loss_mean.item(),2)}, \
     Loss_min: {round(train_loss_min.item(),2)}, Loss_max: {round(train_loss_max.item(),2)} \n \
     Loss_mean_val: {round(test_loss_mean.item(),2)}, Loss_min_val: {round(test_loss_min.item(),2)}, \
     Loss_max_val: {round(test_loss_max.item(),2)} \n \
     Loss_mean_trainval: {round(trainval_loss_mean.item(),2)}')

    # Save the model if the current test loss is the best seen so far
    if test_loss_mean < best_val_loss:
        best_val_epoch = epoch+1
        best_val_loss = test_loss_mean
        best_val_model = model.state_dict()
    if train_loss_mean < best_train_loss:
        best_train_epoch = epoch+1
        best_train_loss = train_loss_mean
        best_train_model = model.state_dict()
    if trainval_loss_mean < best_trainval_loss:
        best_trainval_epoch = epoch+1
        best_trainval_loss = trainval_loss_mean
        best_trainval_model = model.state_dict()
        
torch.save(best_val_model, best_val_model_path)
print(f"Model saved with the best val loss. \n Epoch: {best_val_epoch}, Loss_val: {round(best_val_loss.item(),2)}")
torch.save(best_train_model, best_train_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_train_epoch}, Loss_val: {round(best_train_loss.item(),2)}")
torch.save(best_trainval_model, best_trainval_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_trainval_epoch}, Loss_val: {round(best_trainval_loss.item(),2)}")

Epoch 1/4000, Loss_mean: 380.16,      Loss_min: 379.02, Loss_max: 381.3 
      Loss_mean_val: 379.64, Loss_min_val: 373.8,      Loss_max_val: 384.03 
      Loss_mean_trainval: 379.66
Epoch 2/4000, Loss_mean: 368.31,      Loss_min: 367.41, Loss_max: 369.2 
      Loss_mean_val: 369.11, Loss_min_val: 363.3,      Loss_max_val: 374.41 
      Loss_mean_trainval: 369.07
Epoch 3/4000, Loss_mean: 357.6,      Loss_min: 355.47, Loss_max: 359.74 
      Loss_mean_val: 359.6, Loss_min_val: 344.86,      Loss_max_val: 371.13 
      Loss_mean_trainval: 359.5
Epoch 4/4000, Loss_mean: 345.86,      Loss_min: 344.43, Loss_max: 347.29 
      Loss_mean_val: 349.45, Loss_min_val: 333.81,      Loss_max_val: 365.65 
      Loss_mean_trainval: 349.27
Epoch 5/4000, Loss_mean: 335.5,      Loss_min: 333.02, Loss_max: 337.99 
      Loss_mean_val: 340.89, Loss_min_val: 316.99,      Loss_max_val: 360.35 
      Loss_mean_trainval: 340.62
Epoch 6/4000, Loss_mean: 325.08,      Loss_min: 323.29, Loss_max: 326.88 
      Los

Epoch 46/4000, Loss_mean: 131.21,      Loss_min: 127.61, Loss_max: 134.81 
      Loss_mean_val: 178.05, Loss_min_val: 116.16,      Loss_max_val: 268.11 
      Loss_mean_trainval: 175.73
Epoch 47/4000, Loss_mean: 129.88,      Loss_min: 129.27, Loss_max: 130.49 
      Loss_mean_val: 175.6, Loss_min_val: 106.85,      Loss_max_val: 255.12 
      Loss_mean_trainval: 173.33
Epoch 48/4000, Loss_mean: 129.99,      Loss_min: 124.93, Loss_max: 135.06 
      Loss_mean_val: 175.32, Loss_min_val: 126.28,      Loss_max_val: 239.83 
      Loss_mean_trainval: 173.07
Epoch 49/4000, Loss_mean: 128.3,      Loss_min: 126.94, Loss_max: 129.67 
      Loss_mean_val: 172.92, Loss_min_val: 124.73,      Loss_max_val: 241.84 
      Loss_mean_trainval: 170.71
Epoch 50/4000, Loss_mean: 128.42,      Loss_min: 97.3, Loss_max: 159.53 
      Loss_mean_val: 172.09, Loss_min_val: 130.73,      Loss_max_val: 227.84 
      Loss_mean_trainval: 169.93
Epoch 51/4000, Loss_mean: 127.14,      Loss_min: 126.1, Loss_max: 128.19 


Epoch 91/4000, Loss_mean: 108.12,      Loss_min: 107.72, Loss_max: 108.51 
      Loss_mean_val: 159.87, Loss_min_val: 95.11,      Loss_max_val: 226.72 
      Loss_mean_trainval: 157.31
Epoch 92/4000, Loss_mean: 118.4,      Loss_min: 86.11, Loss_max: 150.69 
      Loss_mean_val: 157.88, Loss_min_val: 101.92,      Loss_max_val: 222.02 
      Loss_mean_trainval: 155.92
Epoch 93/4000, Loss_mean: 107.29,      Loss_min: 105.83, Loss_max: 108.75 
      Loss_mean_val: 159.8, Loss_min_val: 90.45,      Loss_max_val: 246.88 
      Loss_mean_trainval: 157.2
Epoch 94/4000, Loss_mean: 108.09,      Loss_min: 99.75, Loss_max: 116.44 
      Loss_mean_val: 160.21, Loss_min_val: 107.84,      Loss_max_val: 216.77 
      Loss_mean_trainval: 157.63
Epoch 95/4000, Loss_mean: 112.7,      Loss_min: 93.51, Loss_max: 131.88 
      Loss_mean_val: 158.27, Loss_min_val: 91.17,      Loss_max_val: 212.99 
      Loss_mean_trainval: 156.01
Epoch 96/4000, Loss_mean: 105.45,      Loss_min: 81.68, Loss_max: 129.21 
      

Epoch 136/4000, Loss_mean: 92.64,      Loss_min: 90.27, Loss_max: 95.01 
      Loss_mean_val: 148.79, Loss_min_val: 96.3,      Loss_max_val: 202.9 
      Loss_mean_trainval: 146.0
Epoch 137/4000, Loss_mean: 91.92,      Loss_min: 91.76, Loss_max: 92.08 
      Loss_mean_val: 149.49, Loss_min_val: 85.89,      Loss_max_val: 195.39 
      Loss_mean_trainval: 146.64
Epoch 138/4000, Loss_mean: 91.98,      Loss_min: 84.71, Loss_max: 99.26 
      Loss_mean_val: 148.02, Loss_min_val: 99.01,      Loss_max_val: 218.01 
      Loss_mean_trainval: 145.24
Epoch 139/4000, Loss_mean: 91.46,      Loss_min: 89.39, Loss_max: 93.53 
      Loss_mean_val: 147.57, Loss_min_val: 77.17,      Loss_max_val: 197.62 
      Loss_mean_trainval: 144.79
Epoch 140/4000, Loss_mean: 90.15,      Loss_min: 87.42, Loss_max: 92.88 
      Loss_mean_val: 147.13, Loss_min_val: 90.78,      Loss_max_val: 220.2 
      Loss_mean_trainval: 144.31
Epoch 141/4000, Loss_mean: 90.22,      Loss_min: 83.9, Loss_max: 96.53 
      Loss_mean_v

Epoch 181/4000, Loss_mean: 80.08,      Loss_min: 67.69, Loss_max: 92.46 
      Loss_mean_val: 142.92, Loss_min_val: 85.49,      Loss_max_val: 217.88 
      Loss_mean_trainval: 139.81
Epoch 182/4000, Loss_mean: 78.84,      Loss_min: 67.9, Loss_max: 89.78 
      Loss_mean_val: 140.22, Loss_min_val: 88.76,      Loss_max_val: 205.31 
      Loss_mean_trainval: 137.17
Epoch 183/4000, Loss_mean: 79.45,      Loss_min: 71.87, Loss_max: 87.02 
      Loss_mean_val: 138.9, Loss_min_val: 95.82,      Loss_max_val: 186.42 
      Loss_mean_trainval: 135.95
Epoch 184/4000, Loss_mean: 77.0,      Loss_min: 73.67, Loss_max: 80.33 
      Loss_mean_val: 138.83, Loss_min_val: 94.59,      Loss_max_val: 179.8 
      Loss_mean_trainval: 135.76
Epoch 185/4000, Loss_mean: 77.46,      Loss_min: 70.59, Loss_max: 84.33 
      Loss_mean_val: 138.57, Loss_min_val: 93.32,      Loss_max_val: 199.83 
      Loss_mean_trainval: 135.54
Epoch 186/4000, Loss_mean: 77.62,      Loss_min: 72.5, Loss_max: 82.73 
      Loss_mean_v

...

Epoch 3783/4000, Loss_mean: 4.73,      Loss_min: 2.88, Loss_max: 6.58 
      Loss_mean_val: 163.77, Loss_min_val: 66.01,      Loss_max_val: 284.71 
      Loss_mean_trainval: 155.89
Epoch 3784/4000, Loss_mean: 4.53,      Loss_min: 3.31, Loss_max: 5.74 
      Loss_mean_val: 157.45, Loss_min_val: 88.51,      Loss_max_val: 260.57 
      Loss_mean_trainval: 149.87
Epoch 3785/4000, Loss_mean: 4.36,      Loss_min: 4.19, Loss_max: 4.53 
      Loss_mean_val: 165.73, Loss_min_val: 64.19,      Loss_max_val: 323.96 
      Loss_mean_trainval: 157.73
Epoch 3786/4000, Loss_mean: 5.0,      Loss_min: 4.83, Loss_max: 5.16 
      Loss_mean_val: 164.76, Loss_min_val: 83.68,      Loss_max_val: 259.67 
      Loss_mean_trainval: 156.84
Epoch 3787/4000, Loss_mean: 4.52,      Loss_min: 3.84, Loss_max: 5.2 
      Loss_mean_val: 163.43, Loss_min_val: 69.03,      Loss_max_val: 325.09 
      Loss_mean_trainval: 155.55
Epoch 3788/4000, Loss_mean: 5.97,      Loss_min: 4.92, Loss_max: 7.01 
      Loss_mean_val: 163.6

Epoch 3829/4000, Loss_mean: 4.17,      Loss_min: 3.02, Loss_max: 5.31 
      Loss_mean_val: 163.46, Loss_min_val: 91.08,      Loss_max_val: 282.42 
      Loss_mean_trainval: 155.56
Epoch 3830/4000, Loss_mean: 4.01,      Loss_min: 3.54, Loss_max: 4.48 
      Loss_mean_val: 163.69, Loss_min_val: 61.74,      Loss_max_val: 296.6 
      Loss_mean_trainval: 155.78
Epoch 3831/4000, Loss_mean: 4.01,      Loss_min: 3.93, Loss_max: 4.09 
      Loss_mean_val: 160.95, Loss_min_val: 70.48,      Loss_max_val: 300.3 
      Loss_mean_trainval: 153.17
Epoch 3832/4000, Loss_mean: 4.14,      Loss_min: 3.05, Loss_max: 5.22 
      Loss_mean_val: 162.81, Loss_min_val: 67.38,      Loss_max_val: 325.08 
      Loss_mean_trainval: 154.95
Epoch 3833/4000, Loss_mean: 3.97,      Loss_min: 3.75, Loss_max: 4.19 
      Loss_mean_val: 164.62, Loss_min_val: 70.01,      Loss_max_val: 286.34 
      Loss_mean_trainval: 156.65
Epoch 3834/4000, Loss_mean: 4.11,      Loss_min: 3.18, Loss_max: 5.04 
      Loss_mean_val: 163.3

Epoch 3875/4000, Loss_mean: 4.11,      Loss_min: 3.18, Loss_max: 5.05 
      Loss_mean_val: 166.98, Loss_min_val: 53.23,      Loss_max_val: 323.88 
      Loss_mean_trainval: 158.91
Epoch 3876/4000, Loss_mean: 3.94,      Loss_min: 3.7, Loss_max: 4.18 
      Loss_mean_val: 170.5, Loss_min_val: 77.75,      Loss_max_val: 294.55 
      Loss_mean_trainval: 162.24
Epoch 3877/4000, Loss_mean: 3.85,      Loss_min: 3.68, Loss_max: 4.01 
      Loss_mean_val: 167.32, Loss_min_val: 69.52,      Loss_max_val: 302.66 
      Loss_mean_trainval: 159.21
Epoch 3878/4000, Loss_mean: 3.99,      Loss_min: 3.31, Loss_max: 4.66 
      Loss_mean_val: 170.05, Loss_min_val: 87.98,      Loss_max_val: 349.89 
      Loss_mean_trainval: 161.82
Epoch 3879/4000, Loss_mean: 3.84,      Loss_min: 3.26, Loss_max: 4.43 
      Loss_mean_val: 167.21, Loss_min_val: 78.03,      Loss_max_val: 341.99 
      Loss_mean_trainval: 159.11
Epoch 3880/4000, Loss_mean: 3.86,      Loss_min: 3.31, Loss_max: 4.4 
      Loss_mean_val: 164.99

Epoch 3921/4000, Loss_mean: 3.71,      Loss_min: 3.42, Loss_max: 4.0 
      Loss_mean_val: 168.23, Loss_min_val: 94.23,      Loss_max_val: 314.37 
      Loss_mean_trainval: 160.07
Epoch 3922/4000, Loss_mean: 3.73,      Loss_min: 3.34, Loss_max: 4.13 
      Loss_mean_val: 166.7, Loss_min_val: 70.37,      Loss_max_val: 346.76 
      Loss_mean_trainval: 158.62
Epoch 3923/4000, Loss_mean: 3.74,      Loss_min: 3.7, Loss_max: 3.79 
      Loss_mean_val: 171.65, Loss_min_val: 78.7,      Loss_max_val: 468.92 
      Loss_mean_trainval: 163.32
Epoch 3924/4000, Loss_mean: 3.73,      Loss_min: 3.53, Loss_max: 3.93 
      Loss_mean_val: 169.65, Loss_min_val: 89.02,      Loss_max_val: 306.38 
      Loss_mean_trainval: 161.42
Epoch 3925/4000, Loss_mean: 3.93,      Loss_min: 2.51, Loss_max: 5.35 
      Loss_mean_val: 171.01, Loss_min_val: 68.24,      Loss_max_val: 317.07 
      Loss_mean_trainval: 162.72
Epoch 3926/4000, Loss_mean: 3.78,      Loss_min: 3.38, Loss_max: 4.18 
      Loss_mean_val: 166.56,

Epoch 3967/4000, Loss_mean: 4.07,      Loss_min: 3.06, Loss_max: 5.08 
      Loss_mean_val: 167.73, Loss_min_val: 77.81,      Loss_max_val: 332.39 
      Loss_mean_trainval: 159.62
Epoch 3968/4000, Loss_mean: 3.69,      Loss_min: 3.24, Loss_max: 4.15 
      Loss_mean_val: 169.88, Loss_min_val: 59.73,      Loss_max_val: 411.01 
      Loss_mean_trainval: 161.64
Epoch 3969/4000, Loss_mean: 3.69,      Loss_min: 3.13, Loss_max: 4.25 
      Loss_mean_val: 167.18, Loss_min_val: 40.95,      Loss_max_val: 352.29 
      Loss_mean_trainval: 159.08
Epoch 3970/4000, Loss_mean: 3.66,      Loss_min: 3.6, Loss_max: 3.73 
      Loss_mean_val: 171.16, Loss_min_val: 69.36,      Loss_max_val: 280.37 
      Loss_mean_trainval: 162.86
Epoch 3971/4000, Loss_mean: 3.67,      Loss_min: 3.46, Loss_max: 3.87 
      Loss_mean_val: 168.09, Loss_min_val: 73.81,      Loss_max_val: 407.06 
      Loss_mean_trainval: 159.94
Epoch 3972/4000, Loss_mean: 3.84,      Loss_min: 3.16, Loss_max: 4.53 
      Loss_mean_val: 167.